In [130]:
import pandas as pd
import requests
from pprint import pprint
from math import ceil
import copy
import random
from datetime import datetime, timedelta

# Supporting Functions

In [13]:
def chunk_list(lst, chunk_size):
    """
    Splits a list into chunks of specified size.
    
    Parameters:
        lst (list): The list to split.
        chunk_size (int): The maximum size of each chunk.
        
    Returns:
        generator: A generator that yields chunks of the list.
    """
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

In [9]:
def get_unified_app_data(api_key, base_url, app_ids):
    """
    Fetch unified app data from the SensorTower API in batches of 100 IDs per request.
    
    Parameters:
        api_key (str): Your SensorTower API key.
        base_url (str): The base URL for the API.
        app_ids (list[str]): List of unified app IDs to fetch.
        
    Returns:
        list[dict]: Combined response data from all requests.
    """
    
    # API endpoint
    endpoint = "/v1/unified/apps"
    url = f"{base_url}{endpoint}"
    
    headers = {
        'Authorization': f'Bearer {api_key}'
    }
    
    all_responses = []
    chunk_size = 100
    count = 1
    
    # Split the list of app_ids into chunks of 100
    for app_id_chunk in chunk_list(app_ids, chunk_size):
        params = {
            'app_id_type': 'unified',
            'app_ids': ','.join(app_id_chunk)
        }
        
        print('making call for chunk number {}'.format(count))
        
        # Make the API request for this chunk
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            print('call successful!')
            all_responses.extend(response.json().get('apps', []))
        else:
            print(f"Request failed with status code {response.status_code}: {response.text}")
        
        count += 1
    
    return all_responses

In [10]:
def extract_app_ids(list_of_dict):
    return list(map(lambda d: d['app_id'], list_of_dict))

In [84]:
def get_app_id_lists(unified_app_data):
#     also known as canonical data, cuz it also contains canonical id thingy
    itunes_app_id_list = []
    android_app_id_list = []
    
    for app in unified_app_data:
        current_app_itunes_app_ids = app['itunes_apps']
        current_app_android_app_ids = app['android_apps']
        itunes_app_id_list.extend(extract_app_ids(current_app_itunes_app_ids))
        android_app_id_list.extend(extract_app_ids(current_app_android_app_ids))
    
#     ios app id list needs to be adjusted (but not for android app id list)
    itunes_app_id_list_adjusted = list(map(lambda x: str(x),itunes_app_id_list))
    
    result_list = []
    
    for app_id in itunes_app_id_list_adjusted:
        result_list.append(
            {
                'os':'ios',
                'app_id':app_id
            }
        
        )

    for app_id in android_app_id_list:
        result_list.append(
            {
                'os':'android',
                'app_id':app_id
            }
        
        )
    
    return result_list

In [18]:
def get_local_app_info(api_key, base_url, os, app_ids):
    
    # API endpoint
    endpoint = "/v1/{}/apps".format(os)
    url = f"{base_url}{endpoint}"
    
    headers = {
        'Authorization': f'Bearer {api_key}'
    }
    
    all_responses = []
    chunk_size = 100
    count = 1
    
    # Split the list of app_ids into chunks of 100
    for app_id_chunk in chunk_list(app_ids, chunk_size):
        params = {
            'app_ids': ','.join(app_id_chunk)
        }
        
        print('making call for chunk number {}'.format(count))
        
        # Make the API request for this chunk
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            print('call successful!')
            all_responses.extend(response.json().get('apps', []))
        else:
            print(f"Request failed with status code {response.status_code}: {response.text}")
        
        count += 1
    
    return all_responses

In [99]:
def get_monthly_app_performance_single_app_id(api_key, base_url, os, app_id, country_code, start_date, end_date):
    """
    Query monthly app performance data in 12-month segments if the range exceeds 12 months.
    """
    
    def split_date_range(start_date, end_date, max_months=12):
        """
        Split a date range into non-overlapping segments of up to `max_months` months.
        """
        current_start = datetime.strptime(start_date, '%Y-%m-%d')
        final_end = datetime.strptime(end_date, '%Y-%m-%d')
        date_ranges = []

        while current_start < final_end:
            # Calculate the end date for the current range
            current_end = current_start + timedelta(days=max_months * 30)  # Approximate 12 months
            # Ensure current_end does not exceed final_end
            current_end = min(current_end, final_end)
            # Adjust current_end to the last day of the month
            current_end = (current_end.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)

            # Add the range to the list
            date_ranges.append((current_start.strftime('%Y-%m-%d'), current_end.strftime('%Y-%m-%d')))

            # Move current_start to the first day of the next month after current_end
            current_start = (current_end + timedelta(days=1)).replace(day=1)

        return date_ranges
    
    print("Getting performance data for app id {}".format(app_id))

    endpoint = "/v1/{}/sales_report_estimates".format(os)
    url = f"{base_url}{endpoint}"

    headers = {
        'Authorization': f'Bearer {api_key}'
    }

    date_ranges = split_date_range(start_date, end_date)
    
#     print("Date Ranges")
#     print(date_ranges)
    
    
    all_results = []

    for start, end in date_ranges:
        params = {
            'os': os,
            'app_ids': app_id,
            'countries': country_code,
            'date_granularity': 'monthly',
            'start_date': start,
            'end_date': end,
        }

        request = requests.Request("GET", url, headers=headers, params=params)
        prepared_request = request.prepare()

#         print("Requesting Data for Range:", start, "to", end)
#         print("Full Request URL:", prepared_request.url)

        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            print(f"Data retrieved for {start} to {end}")
            json_data = response.json()
            if isinstance(json_data, list):  # If the response is a list
                all_results.extend(json_data)
            elif isinstance(json_data, dict):  # If the response is a dictionary
                all_results.extend(json_data.get('data', []))
            else:
                print(f"Unexpected response format for {start} to {end}: {json_data}")
        else:
            print(f"Failed for {start} to {end}. Status Code: {response.status_code}")
            print(response.text)
    
    return all_results


In [113]:
def get_performance_streams_from_list_of_app_ids(api_key, base_url, country_code, start_date, end_date, list_app_ids):
    
    result = []
    
    for app in list_app_ids:
        os = app["os"]
        app_id = app["app_id"]
        app_performance = get_monthly_app_performance_single_app_id(
            api_key, 
            base_url, 
            os, 
            app_id, 
            country_code,
            start_date, 
            end_date
        )
        result.append(
            {
                'app_id': app_id,
                'os': os,
                'app_performance': app_performance
            }
        )

    return result

In [114]:
def get_unique_dates(performance_streams_data):
    result = []
    for app in performance_streams_data:
        for record in app['app_performance']:
            if not(record['d'] in result):
                result.append(record['d'])
    return sorted(result)

In [120]:
def get_app_ids_from_performance_stream_data(performance_stream_data):
    return list(map(lambda x: x['app_id'], performance_stream_data))

# API Credentials

In [54]:
api_key = 'ST0_ejEo8DwM_vCjQJzNhPbpmhQ'
base_url = 'https://api.sensortower.com'

# using authentication token from chi Tram's account

# Load st data file

In [3]:
# Load data
raw_game_data_df = pd.read_excel("data/output/st_game_data_2014_2024.xlsx")

In [4]:
raw_game_data_df

,Unified Name,Unified ID,Unified Publisher Name,Unified Publisher ID,Date,Platform,Category,Downloads (Absolute),Downloads (Growth),Downloads (Growth %),"Revenue (Absolute, $)","Revenue (Growth, $)",Revenue (Growth %),Average DAU (Absolute),Average DAU (Growth),Average DAU (Growth %)
0,Vermillion Watch: Moorgate Accord - Hidden Obj...,58cb5ba50211a68f8e00007e,Pixel United,56289df502ac6486a7009de2,2024-01-01,Unified,Games,0,0,NaN,38.26,-7.90,-0.17,NaN,NaN,NaN
1,SSC '22 - Super Soccer Champs,5cf1572b49a1a977dbf7f9f8,CMA Megacorp,560c7c878ac350643902ef59,2024-01-01,Unified,Games,2034,-1546,-0.43,1423.87,-1007.01,-0.41,15.0,-15.0,-0.50
2,Sphinx Trivia - Win Real Cash,5b5e6a963d7e3c2daafcc304,GMRD Apps,5ee985beaf4b673c0e63d3df,2024-01-01,Unified,Games,7,6,6.00,1416.69,-183.91,-0.11,NaN,NaN,NaN
3,Hero Realms,629e1bd818009713fd2efc35,Wise Wizard Games,61e9b3ce6d8df53175fcc9ee,2024-01-01,Unified,Games,0,-5,-1.00,1418.47,854.43,1.51,NaN,NaN,NaN
4,Hexapolis: Civilization wars,605587da4ebdbc721f470cb6,Jiri Bukovjan,55f896028ac350426b044f4d,2024-01-01,Unified,Strategy,4305,-5224,-0.55,1418.83,593.34,0.72,868.0,241.0,0.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97270,Resimli Kelime Bulmaca,55d20e5b02ac64ee730006cb,Bretzel Games,5cc750557927800edc53a6ce,2014-01-01,Unified,Word,110,110,NaN,3.95,3.95,NaN,NaN,NaN,NaN
97271,Black Metal Man,537b76ff830f782dbe00c0f9,SinSquid,5604943f8ac350172200fddc,2014-01-01,Unified,Music,0,0,NaN,3.94,3.94,NaN,NaN,NaN,NaN
97272,Эврика! Логические Задачи Игры,55d20e5702ac64ee730005ef,Nekhoroshev Ilya,55f893ad8ac350426b039896,2014-01-01,Unified,Trivia,0,0,NaN,3.93,3.93,NaN,NaN,NaN,NaN
97273,Cotton Candy! - Maker Games,55d93a3b02ac645ad203e6ef,Bluebear Technologies,5614dc003f07e25d29019275,2014-01-01,Unified,Games,4295,-3928,-0.48,3.93,-45.88,-0.92,NaN,NaN,NaN


# Use Rise of Kingdoms as the test game

In [6]:
raw_game_data_df[raw_game_data_df["Unified Name"] == "Rise of Kingdoms"]

,Unified Name,Unified ID,Unified Publisher Name,Unified Publisher ID,Date,Platform,Category,Downloads (Absolute),Downloads (Growth),Downloads (Growth %),"Revenue (Absolute, $)","Revenue (Growth, $)",Revenue (Growth %),Average DAU (Absolute),Average DAU (Growth),Average DAU (Growth %)
3337,Rise of Kingdoms,5ac2bdddcfc03208313848db,Lilith 莉莉丝,59418862660953716600e6f7,2024-01-01,Unified,Games,738234,-70916,-0.09,3058963.86,-2076873.90,-0.40,85507.0,-28668.0,-0.25
10000,Rise of Kingdoms,5ac2bdddcfc03208313848db,Lilith 莉莉丝,59418862660953716600e6f7,2023-01-01,Unified,Strategy,964015,-2295941,-0.70,5892819.33,-2602197.56,-0.31,116927.0,-33522.0,-0.22
26664,Rise of Kingdoms,5ac2bdddcfc03208313848db,Lilith 莉莉丝,59418862660953716600e6f7,2022-01-01,Unified,Games,3259956,3259956,NaN,8495016.89,8495016.89,NaN,150449.0,150449.0,NaN
37320,Rise of Kingdoms,5ac2bdddcfc03208313848db,Lilith 莉莉丝,59418862660953716600e6f7,2021-01-01,Unified,Games,323301,-2025318,-0.86,4191293.14,-6048204.35,-0.59,109406.0,-109996.0,-0.50
46069,Rise of Kingdoms,5ac2bdddcfc03208313848db,Lilith 莉莉丝,59418862660953716600e6f7,2020-01-01,Unified,Strategy,2348619,854175,0.57,10239497.49,6808345.39,1.98,219443.0,124140.0,1.30
53985,Rise of Kingdoms,5ac2bdddcfc03208313848db,Lilith 莉莉丝,59418862660953716600e6f7,2019-01-01,Unified,Games,1494444,1004453,2.05,3431152.10,2957736.96,6.25,95347.0,76264.0,4.00
66676,Rise of Kingdoms,5ac2bdddcfc03208313848db,Lilith 莉莉丝,59418862660953716600e6f7,2018-01-01,Unified,Strategy,489991,489991,NaN,473415.14,473415.14,NaN,19083.0,19083.0,NaN


In [7]:
rok_unified_id = "5ac2bdddcfc03208313848db"

In [57]:
# Another test case - Mobile Legends Bang Bang

In [58]:
raw_game_data_df[raw_game_data_df["Unified Name"] == "Mobile Legends: Bang Bang"]

,Unified Name,Unified ID,Unified Publisher Name,Unified Publisher ID,Date,Platform,Category,Downloads (Absolute),Downloads (Growth),Downloads (Growth %),"Revenue (Absolute, $)","Revenue (Growth, $)",Revenue (Growth %),Average DAU (Absolute),Average DAU (Growth),Average DAU (Growth %)
3299,Mobile Legends: Bang Bang,57955d280211a6718a000002,Moonton 沐瞳,62c4b457b3ae271ff6cc58a4,2024-01-01,Unified,Action,3341367,1908518,1.33,619308.61,149651.89,0.32,647957.0,-71868.0,-0.10
16688,Mobile Legends: Bang Bang,57955d280211a6718a000002,Moonton 沐瞳,62c4b457b3ae271ff6cc58a4,2023-01-01,Unified,Action,1660020,-1022923,-0.38,545134.67,38457.72,0.08,747450.0,-202413.0,-0.21
26716,Mobile Legends: Bang Bang,57955d280211a6718a000002,Moonton 沐瞳,62c4b457b3ae271ff6cc58a4,2022-01-01,Unified,Action,2682943,-715444,-0.21,506676.95,-26647.66,-0.05,949863.0,114761.0,0.14
37331,Mobile Legends: Bang Bang,57955d280211a6718a000002,Moonton 沐瞳,62c4b457b3ae271ff6cc58a4,2021-01-01,Unified,Action,3398387,-1212747,-0.26,533324.61,88274.23,0.20,835102.0,113208.0,0.16
46709,Mobile Legends: Bang Bang,57955d280211a6718a000002,Moonton 沐瞳,62c4b457b3ae271ff6cc58a4,2020-01-01,Unified,Action,4611134,-3006325,-0.39,445050.38,10956.14,0.03,721362.0,203773.0,0.39
53953,Mobile Legends: Bang Bang,57955d280211a6718a000002,Moonton 沐瞳,62c4b457b3ae271ff6cc58a4,2019-01-01,Unified,Action,7617459,5380125,2.40,434094.24,397793.32,10.96,518048.0,484510.0,14.45
66659,Mobile Legends: Bang Bang,57955d280211a6718a000002,Moonton 沐瞳,62c4b457b3ae271ff6cc58a4,2018-01-01,Unified,Action,6948781,2507845,0.56,422070.67,242829.81,1.35,749404.0,260254.0,0.53
72826,Mobile Legends: Bang Bang,57955d280211a6718a000002,Moonton 沐瞳,62c4b457b3ae271ff6cc58a4,2017-01-01,Unified,Action,4440936,2242163,1.02,179240.86,165464.34,12.01,489150.0,415207.0,5.62
84287,Mobile Legends: Bang Bang,57955d280211a6718a000002,Moonton 沐瞳,62c4b457b3ae271ff6cc58a4,2016-01-01,Unified,Action,2198773,2198773,NaN,13776.52,13776.52,NaN,73741.0,73741.0,NaN


In [59]:
mlbb_unified_id = "57955d280211a6718a000002"

# Get all the local app id of this game

In [14]:
# Get unified app data first

unified_app_data = get_unified_app_data(
    api_key, base_url, ["5ac2bdddcfc03208313848db",]
)

making call for chunk number 1
call successful!


In [15]:
unified_app_data

[{'unified_app_id': '5ac2bdddcfc03208313848db',
  'name': 'Rise of Kingdoms',
  'canonical_app_id': 1354260888,
  'cohort_id': 'cohort_e98ab6fcac86cc1e1b6163cd',
  'itunes_apps': [{'app_id': 1354260888},
   {'app_id': 1519945981},
   {'app_id': 1483025097},
   {'app_id': 1474599374},
   {'app_id': 1436964519},
   {'app_id': 1574489575}],
  'android_apps': [{'app_id': 'com.lilithgame.roc.gp'},
   {'app_id': 'com.lilithgames.rok.gpkr'},
   {'app_id': 'com.lilithgames.rok.gp.jp'},
   {'app_id': 'com.lilithgame.roc.gp.tw'},
   {'app_id': 'com.rok.gp.vn'}],
  'unified_publisher_ids': ['59418862660953716600e6f7'],
  'itunes_publisher_ids': [982119759,
   1481453487,
   982163649,
   1478872809,
   1377104229],
  'android_publisher_ids': ['LilithGames', '奧瑞吉有限公司']}]

In [16]:
# Get app_id list

app_id_dict = get_app_id_lists(unified_app_data)

In [17]:
app_id_dict

{'ios_app_id_list': ['1354260888',
  '1519945981',
  '1483025097',
  '1474599374',
  '1436964519',
  '1574489575'],
 'android_app_id_list': ['com.lilithgame.roc.gp',
  'com.lilithgames.rok.gpkr',
  'com.lilithgames.rok.gp.jp',
  'com.lilithgame.roc.gp.tw',
  'com.rok.gp.vn']}

In [60]:
# Second test case - MLBB

In [61]:
unified_app_data = get_unified_app_data(
    api_key, base_url, ["57955d280211a6718a000002",]
)

making call for chunk number 1
call successful!


In [62]:
unified_app_data

[{'unified_app_id': '57955d280211a6718a000002',
  'name': 'Mobile Legends: Bang Bang',
  'canonical_app_id': 'com.mobile.legends',
  'cohort_id': 'cohort_62c5c2137664b3a4fd1a03ce',
  'itunes_apps': [{'app_id': 1160056295},
   {'app_id': 1428475506},
   {'app_id': 6468942281},
   {'app_id': 6670476964}],
  'android_apps': [{'app_id': 'com.mobile.legends'},
   {'app_id': 'com.vng.mlbbvn'},
   {'app_id': 'com.mobiin.gp'},
   {'app_id': 'com.hhgame.mlbbvn'}],
  'unified_publisher_ids': ['562890fe02ac648b2800231f',
   '5bc8740ca2750e2e014a1c19',
   '61176426ca971556e3a545d2',
   '62c4b457b3ae271ff6cc58a4',
   '653425cf80474a233591b66b',
   '66054d3dedca1062d05c2a61'],
  'itunes_publisher_ids': [910435102, 398453277, 1438383758, 1706091894],
  'android_publisher_ids': ['Moonton',
   'VNG+Corporation+-+Công+ty+Cổ+phần+VNG',
   'Vizta+Games',
   'Công+ty+cổ+phần+phát+triển+công+nghệ+số+Hồng+Hà']}]

In [85]:
app_id_list = get_app_id_lists(unified_app_data)

In [86]:
app_id_list

[{'os': 'ios', 'app_id': '1160056295'},
 {'os': 'ios', 'app_id': '1428475506'},
 {'os': 'ios', 'app_id': '6468942281'},
 {'os': 'ios', 'app_id': '6670476964'},
 {'os': 'android', 'app_id': 'com.mobile.legends'},
 {'os': 'android', 'app_id': 'com.vng.mlbbvn'},
 {'os': 'android', 'app_id': 'com.mobiin.gp'},
 {'os': 'android', 'app_id': 'com.hhgame.mlbbvn'}]

# Get all vn revenue streams of these local app id from the earliest possible date til now

In [95]:
rev_stream_test_rok = get_monthly_app_performance_single_app_id(
    api_key, 
    base_url, 
    os = 'ios', 
    app_id = '1160056295', 
    country_code = 'VN', 
    start_date = '2014-01-01', 
    end_date = '2024-10-31'
)

Getting performance data for app id 1160056295
Date Ranges
[('2014-01-01', '2014-12-27'), ('2014-12-28', '2015-12-23'), ('2015-12-24', '2016-12-18'), ('2016-12-19', '2017-12-14'), ('2017-12-15', '2018-12-10'), ('2018-12-11', '2019-12-06'), ('2019-12-07', '2020-12-01'), ('2020-12-02', '2021-11-27'), ('2021-11-28', '2022-11-23'), ('2022-11-24', '2023-11-19'), ('2023-11-20', '2024-10-31')]
Data retrieved for 2014-01-01 to 2014-12-27
Data retrieved for 2014-12-28 to 2015-12-23
Data retrieved for 2015-12-24 to 2016-12-18
Data retrieved for 2016-12-19 to 2017-12-14
Data retrieved for 2017-12-15 to 2018-12-10
Data retrieved for 2018-12-11 to 2019-12-06
Data retrieved for 2019-12-07 to 2020-12-01
Data retrieved for 2020-12-02 to 2021-11-27
Data retrieved for 2021-11-28 to 2022-11-23
Data retrieved for 2022-11-24 to 2023-11-19
Data retrieved for 2023-11-20 to 2024-10-31


In [81]:
rev_stream_test_rok

[{'aid': 1574489575,
  'cc': 'VN',
  'd': '2022-01-01T00:00:00Z',
  'au': 23086,
  'ar': 460301,
  'iu': 102361,
  'ir': 10080465},
 {'aid': 1574489575,
  'cc': 'VN',
  'd': '2022-02-01T00:00:00Z',
  'au': 25966,
  'ar': 1635748,
  'iu': 136643,
  'ir': 37475506},
 {'aid': 1574489575,
  'cc': 'VN',
  'd': '2022-03-01T00:00:00Z',
  'au': 16315,
  'ar': 2161656,
  'iu': 107710,
  'ir': 51266125},
 {'aid': 1574489575,
  'cc': 'VN',
  'd': '2022-04-01T00:00:00Z',
  'au': 8576,
  'ar': 1978652,
  'iu': 58726,
  'ir': 59674095},
 {'aid': 1574489575,
  'cc': 'VN',
  'd': '2022-05-01T00:00:00Z',
  'au': 5727,
  'ar': 2804362,
  'iu': 40891,
  'ir': 59764065},
 {'aid': 1574489575,
  'cc': 'VN',
  'd': '2022-06-01T00:00:00Z',
  'au': 5750,
  'ar': 2227976,
  'iu': 49684,
  'ir': 51264123},
 {'aid': 1574489575,
  'cc': 'VN',
  'd': '2022-07-01T00:00:00Z',
  'au': 5360,
  'ar': 2664741,
  'iu': 43381,
  'ir': 29318126},
 {'aid': 1574489575,
  'cc': 'VN',
  'd': '2022-08-01T00:00:00Z',
  'au': 4444

In [88]:
# MLBB

In [100]:
performance_streams_mlbb = get_performance_streams_from_list_of_app_ids(
    api_key, 
    base_url, 
    country_code = "VN", 
    start_date = "2014-01-01", 
    end_date = "2024-10-31", 
    list_app_ids = app_id_list)

Getting performance data for app id 1160056295
Data retrieved for 2014-01-01 to 2014-12-31
Data retrieved for 2015-01-01 to 2015-12-31
Data retrieved for 2016-01-01 to 2016-12-31
Data retrieved for 2017-01-01 to 2017-12-31
Data retrieved for 2018-01-01 to 2018-12-31
Data retrieved for 2019-01-01 to 2019-12-31
Data retrieved for 2020-01-01 to 2020-12-31
Data retrieved for 2021-01-01 to 2021-12-31
Data retrieved for 2022-01-01 to 2022-12-31
Data retrieved for 2023-01-01 to 2023-12-31
Data retrieved for 2024-01-01 to 2024-10-31
Getting performance data for app id 1428475506
Data retrieved for 2014-01-01 to 2014-12-31
Data retrieved for 2015-01-01 to 2015-12-31
Data retrieved for 2016-01-01 to 2016-12-31
Data retrieved for 2017-01-01 to 2017-12-31
Data retrieved for 2018-01-01 to 2018-12-31
Data retrieved for 2019-01-01 to 2019-12-31
Data retrieved for 2020-01-01 to 2020-12-31
Data retrieved for 2021-01-01 to 2021-12-31
Data retrieved for 2022-01-01 to 2022-12-31
Data retrieved for 2023-01

In [101]:
performance_streams_mlbb

[{'app_id': '1160056295',
  'os': 'ios',
  'app_performance': [{'aid': 1160056295,
    'cc': 'VN',
    'd': '2016-11-01T00:00:00Z',
    'au': 24985,
    'ar': 50310,
    'iu': 78417,
    'ir': 75420},
   {'aid': 1160056295,
    'cc': 'VN',
    'd': '2016-12-01T00:00:00Z',
    'au': 10514,
    'ar': 69644,
    'iu': 34589,
    'ir': 185899},
   {'aid': 1160056295,
    'cc': 'VN',
    'd': '2017-01-01T00:00:00Z',
    'au': 12067,
    'ar': 153764,
    'iu': 41489,
    'ir': 153439},
   {'aid': 1160056295,
    'cc': 'VN',
    'd': '2017-02-01T00:00:00Z',
    'au': 7869,
    'ar': 54206,
    'iu': 34086,
    'ir': 278801},
   {'aid': 1160056295,
    'cc': 'VN',
    'd': '2017-03-01T00:00:00Z',
    'au': 8853,
    'ar': 55413,
    'iu': 41640,
    'ir': 258177},
   {'aid': 1160056295,
    'cc': 'VN',
    'd': '2017-04-01T00:00:00Z',
    'au': 9949,
    'ar': 62824,
    'iu': 46050,
    'ir': 364728},
   {'aid': 1160056295,
    'cc': 'VN',
    'd': '2017-05-01T00:00:00Z',
    'au': 15253,
  

In [125]:
list(filter(lambda x: x['app_id'] == 'com.hhgame.mlbbvn', performance_streams_mlbb))

[{'app_id': 'com.hhgame.mlbbvn',
  'os': 'android',
  'app_performance': [{'aid': 'com.hhgame.mlbbvn',
    'c': 'VN',
    'd': '2024-02-01T00:00:00Z',
    'u': 124991,
    'r': 2521078},
   {'aid': 'com.hhgame.mlbbvn',
    'c': 'VN',
    'd': '2024-03-01T00:00:00Z',
    'u': 198971,
    'r': 4628527},
   {'aid': 'com.hhgame.mlbbvn',
    'c': 'VN',
    'd': '2024-04-01T00:00:00Z',
    'u': 261751,
    'r': 5161661},
   {'aid': 'com.hhgame.mlbbvn',
    'c': 'VN',
    'd': '2024-05-01T00:00:00Z',
    'u': 257491,
    'r': 4660375},
   {'aid': 'com.hhgame.mlbbvn',
    'c': 'VN',
    'd': '2024-06-01T00:00:00Z',
    'u': 311127,
    'r': 4305700},
   {'aid': 'com.hhgame.mlbbvn',
    'c': 'VN',
    'd': '2024-07-01T00:00:00Z',
    'u': 421596,
    'r': 4871071},
   {'aid': 'com.hhgame.mlbbvn',
    'c': 'VN',
    'd': '2024-08-01T00:00:00Z',
    'u': 352100,
    'r': 3921492},
   {'aid': 'com.hhgame.mlbbvn',
    'c': 'VN',
    'd': '2024-09-01T00:00:00Z',
    'u': 417389,
    'r': 3386111},
 

# Turn performance stream list of dictionary to dataframe table form

In [103]:
# Get all the unique dates

In [115]:
unique_dates = get_unique_dates(performance_streams_mlbb)

In [118]:
app_ids = get_app_ids_from_performance_stream_data(performance_streams_mlbb)

In [123]:
df_performance_streams_mlbb = pd.DataFrame(index=app_ids, columns=unique_dates)

In [126]:
def fill_performance_streams_df(blank_df, performance_streams_data):
    for row in performance_streams_data:
        if row['os'] == 'ios':
            for record in row['app_performance']:        
                blank_df.at[row['app_id'], record['d']] = record['ar'] + record['ir']
        elif row['os'] == 'android':
            for record in row['app_performance']:        
                blank_df.at[row['app_id'], record['d']] = record['r']

In [124]:
df_performance_streams_mlbb

,2016-09-01T00:00:00Z,2016-10-01T00:00:00Z,2016-11-01T00:00:00Z,2016-12-01T00:00:00Z,2017-01-01T00:00:00Z,2017-02-01T00:00:00Z,2017-03-01T00:00:00Z,2017-04-01T00:00:00Z,2017-05-01T00:00:00Z,2017-06-01T00:00:00Z,...,2024-01-01T00:00:00Z,2024-02-01T00:00:00Z,2024-03-01T00:00:00Z,2024-04-01T00:00:00Z,2024-05-01T00:00:00Z,2024-06-01T00:00:00Z,2024-07-01T00:00:00Z,2024-08-01T00:00:00Z,2024-09-01T00:00:00Z,2024-10-01T00:00:00Z
1160056295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1428475506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6468942281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6670476964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
com.mobile.legends,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
com.vng.mlbbvn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
com.mobiin.gp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
com.hhgame.mlbbvn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
fill_performance_streams_df(df_performance_streams_mlbb, performance_streams_mlbb)

In [128]:
df_performance_streams_mlbb

,2016-09-01T00:00:00Z,2016-10-01T00:00:00Z,2016-11-01T00:00:00Z,2016-12-01T00:00:00Z,2017-01-01T00:00:00Z,2017-02-01T00:00:00Z,2017-03-01T00:00:00Z,2017-04-01T00:00:00Z,2017-05-01T00:00:00Z,2017-06-01T00:00:00Z,...,2024-01-01T00:00:00Z,2024-02-01T00:00:00Z,2024-03-01T00:00:00Z,2024-04-01T00:00:00Z,2024-05-01T00:00:00Z,2024-06-01T00:00:00Z,2024-07-01T00:00:00Z,2024-08-01T00:00:00Z,2024-09-01T00:00:00Z,2024-10-01T00:00:00Z
1160056295,NaN,NaN,125730,255543,307203,333007,313590,427552,628594,524449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1428475506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1449301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6468942281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,163068,1179862,1646898,1579988,1995418,2124653,1502100,1581695,1724824
6670476964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
com.mobile.legends,457,175431,337236,483255,619205,640954,654582,960917,1119389,1210556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
com.vng.mlbbvn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2863411,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
com.mobiin.gp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
com.hhgame.mlbbvn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2521078,4628527,5161661,4660375,4305700,4871071,3921492,3386111,5735495


In [129]:
df_performance_streams_mlbb.to_excel('data/output/df_performance_streams_mlbb.xlsx', index=True)

In [145]:
# Now when each game is representaed by a table of revenue stream, 
# can just multiply the row by respective revenue multiplier of the app id

# Load revenue multiplier file

In [134]:
data_types = {
    "cleaned_publisher_name": "string",
    "publisher_id": "string",
    "publisher_name": "string",
    "revenue_multiplier": "int64"
}

In [135]:
df_revenue_multiplier = pd.read_csv(
    "data/revenue_multiplier.csv",
    dtype = data_types
)

In [136]:
df_revenue_multiplier

,cleaned_publisher_name,publisher_id,publisher_name,revenue_multiplier
0,CMNOnline,1490674259,CMNOnline,1
1,CMNOnline,CMN+G1,CMN G1,1
2,CMNOnline,CMN+Online,CMN Online,1
3,CTCP Giai Tri Thien Thuong Hoa,1580495122,THIEN THUONG HOA ENTERTAINMENT JOINT STOCK COM...,3
4,CTCP Giai Tri Thien Thuong Hoa,1604321544,Watt Games,3
...,...,...,...,...
131,VTC,"VTC+Online,.JSC","VTC Online,.JSC",3
132,ZingPlay,1600022992,ZingPlay VietNam,20
133,ZingPlay,VNG+ZingPlay+Game+Studios,VNG ZingPlay Game Studios,20
134,ZingPlay,VNG+ZingPlay+Studio,VNG ZingPlay Studio,20


In [138]:
# Create a dictionary of publisher_id - multiplier

In [141]:
dict_publisher_id_revenue_multiplier = dict()

for index, row in df_revenue_multiplier.iterrows():
    dict_publisher_id_revenue_multiplier[row['publisher_id']] = row['revenue_multiplier']

In [142]:
dict_publisher_id_revenue_multiplier

{'1490674259': 1,
 'CMN+G1': 1,
 'CMN+Online': 1,
 '1580495122': 3,
 '1604321544': 3,
 '1646907857': 3,
 '1689532424': 3,
 'CHUANG+COOL+ENTERTAINMENT': 3,
 'TTH+GAME': 3,
 'TTH+GAMES': 3,
 'TTH+PLAY': 3,
 'TTH+PLAY+COMPANY+LIMITED': 3,
 'TTHmobi': 3,
 'TTHOA+GAMES': 3,
 'TTHOAGames': 3,
 '959827394': 3,
 'DZOGAME+Viet+Nam': 3,
 '1091188713': 3,
 '1473783022': 3,
 '1682431193': 3,
 'Công+ty+cổ+phần+Funtap': 3,
 'Funtap.vn': 3,
 'Funtap++Games': 3,
 '790194591': 3,
 '896628171': 3,
 '982163649': 3,
 '1418348199': 3,
 '1547396692': 3,
 '1549148665': 3,
 '1676603715': 3,
 '4GAMOTA': 3,
 'APPOTA+INC': 3,
 'GAMOTA+INC': 3,
 'HTD+Media+VN': 3,
 'LilithGames': 3,
 '513828660': 15,
 '1047094789': 15,
 '1096472396': 15,
 '1335627134': 15,
 'Garena+Games+Online': 15,
 'Garena+International+I': 15,
 'Garena+Mobile+Private': 15,
 '1475616011': 3,
 '1547943738': 3,
 'GOSU+Jsc': 3,
 'GOSU+ONLINE+CORPORATION': 3,
 'GOSU+Publisher': 3,
 'Mobi389+-+GOSU': 3,
 '1438383758': 3,
 '1607282322': 3,
 '1698482

In [143]:
len(dict_publisher_id_revenue_multiplier)

136